In [1]:
from y0.dsl import One, P, A, B, C, D, Q, S, T, W, X, Y, Z, Sum

In [2]:
One()

In [3]:
P(A)

P(Distribution(children=(Variable(name='A'),), parents=()))

In [4]:
P(A @ W)

P(Distribution(children=(CounterfactualVariable(name='A', interventions=(Intervention(name='W', star=False),)),), parents=()))

In [5]:
P(A @ ~W)

P(Distribution(children=(CounterfactualVariable(name='A', interventions=(Intervention(name='W', star=True),)),), parents=()))

In [6]:
P(A @ ~W @ X)

P(Distribution(children=(CounterfactualVariable(name='A', interventions=(Intervention(name='W', star=True), Intervention(name='X', star=False))),), parents=()))

In [7]:
P(A, B)

P(Distribution(children=(Variable(name='A'), Variable(name='B')), parents=()))

In [8]:
P(A | B)

P(Distribution(children=(Variable(name='A'),), parents=(Variable(name='B'),)))

In [9]:
One() / P(A | B)

Fraction(numerator=<y0.dsl.One object at 0x7f9bf8fe8070>, denominator=P(Distribution(children=(Variable(name='A'),), parents=(Variable(name='B'),))))

In [10]:
(One() / P(A | B)) * P(A)

Fraction(numerator=P(Distribution(children=(Variable(name='A'),), parents=())), denominator=P(Distribution(children=(Variable(name='A'),), parents=(Variable(name='B'),))))

In [11]:
One() * P(A | B)

P(Distribution(children=(Variable(name='A'),), parents=(Variable(name='B'),)))

In [12]:
P(A | B) * P(B)

Product(expressions=(P(Distribution(children=(Variable(name='A'),), parents=(Variable(name='B'),))), P(Distribution(children=(Variable(name='B'),), parents=()))))

In [13]:
Sum[B](P(A | B))

Sum(expression=P(Distribution(children=(Variable(name='A'),), parents=(Variable(name='B'),))), ranges=(Variable(name='B'),))

In [14]:
Sum(P(A | B) * Sum(P(C | D), [Q]), [S, T])

Sum(expression=Product(expressions=(P(Distribution(children=(Variable(name='A'),), parents=(Variable(name='B'),))), Sum(expression=P(Distribution(children=(Variable(name='C'),), parents=(Variable(name='D'),))), ranges=[Variable(name='Q')]))), ranges=[Variable(name='S'), Variable(name='T')])

In [15]:
Sum[W](P((Y @ ~Z @ W) & X) * P(D) * P(Z @ D) * P(W @ ~X))

Sum(expression=Product(expressions=(P(Distribution(children=(CounterfactualVariable(name='Y', interventions=(Intervention(name='Z', star=True), Intervention(name='W', star=False))), Variable(name='X')), parents=())), P(Distribution(children=(Variable(name='D'),), parents=())), P(Distribution(children=(CounterfactualVariable(name='Z', interventions=(Intervention(name='D', star=False),)),), parents=())), P(Distribution(children=(CounterfactualVariable(name='W', interventions=(Intervention(name='X', star=True),)),), parents=())))), ranges=(Variable(name='W'),))

In [16]:
(P(A) / P(B)) / (P(C) / P(D))

Fraction(numerator=Product(expressions=(P(Distribution(children=(Variable(name='A'),), parents=())), P(Distribution(children=(Variable(name='D'),), parents=())))), denominator=Product(expressions=(P(Distribution(children=(Variable(name='B'),), parents=())), P(Distribution(children=(Variable(name='C'),), parents=())))))